## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Imports

In [1]:
import numpy as np
import cv2
import glob
import pickle
import collections
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os

%matplotlib inline

## Globals

In [2]:
# Directory pathes
path_camera_calibration = './camera_cal/'
path_test_images = './test_images/'
path_ouput_images = './output_images/'

# Pickle file
pickle_file = path_camera_calibration + 'calibration_pickle.p'

# Video files
input_video = 'project_video.mp4'
output_video = 'output_video.mp4'

# Arrays to store object points and image points from all the images.
obj_points = [] # 3d points in real world space
img_points = [] # 2d points in image plane.

# Chessboard dimensions
dimensions = (9, 6)

# Make a list of calibration images
images_list = glob.glob(path_camera_calibration + 'calibration*.jpg')
test_images_list = glob.glob(path_test_images + 'test*.jpg')

# Dictionary for creating binary file for calibration data
dist_pickle = {}

# Mask/Warp values
top_left = [580, 450]
top_right = [720, 450]
bottom_left = [190, 720]
bottom_right = [1190, 720]

proj_top_left = [320, 0]
proj_top_right = [1000, 0]
proj_bottom_left = [320, 720]
proj_bottom_right = [1000, 720]

# Conversion ration from pixel to meters
conv_y = 30/720 # meters per pixel in y dimension
conv_x = 3.7/700 # meters per pixel in x dimension
    
# Stores the previous frame
previous_frame = None

# Check for camera calibration directory
if not os.path.exists(path_camera_calibration):
    os.makedirs(path_camera_calibration)
    
# Check for test images directory
if not os.path.exists(path_test_images):
    os.makedirs(path_test_images)

# Check for output images directory
if not os.path.exists(path_ouput_images):
    os.makedirs(path_ouput_images)

## Helpers

In [3]:
def get_output_file_path(filename):
    '''
    Creates output path for images and returns it.
    '''
    
    return path_ouput_images + filename + '.jpg'

def save_output_file(img, filename):
    '''
    Saves the image as file.
    '''
    
    cv2.imwrite(get_output_file_path(filename), img)

## Camera Calibration

In [4]:
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_p = np.zeros((6*9, 3), np.float32)
obj_p[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images_list):
    # Read each image
    img = cv2.imread(fname)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, dimensions, None)

    # If corners are found, add object points, image points and image to images array
    if ret == True:
        obj_points.append(obj_p)
        img_points.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, dimensions, corners, ret)
        save_output_file(img, 'corners_found' + str(idx))

# Load image for reference
img = cv2.imread(path_camera_calibration + 'calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_size, None, None)

In [5]:
# Save the camra calibration result for later use
dist_pickle = {}
dist_pickle['mtx'] = mtx
dist_pickle['dist'] = dist
pickle.dump(dist_pickle, open(pickle_file, 'wb'))

## Frame Processing

In [6]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    '''
    Applies absolute value of the output from Sobel Operator.
    
    Creates binary image based on given thresholds.
    '''
    
    # Perform gray scaling
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    
    # Apply Sobel
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    
    # Create binary image
    binary = np.zeros_like(scaled_sobel)
    binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary


def color_thresh(image, s_thresh=(0, 255), v_thresh=(0, 255)):
    '''
    Applies color space transformations.
    
    Creates binary image based on given thresholds.
    '''
    # Use HLS space
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # use HSV space
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:, :, 2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= v_thresh[0]) & (v_channel <= v_thresh[1])] = 1

    # Create binary image
    binary = np.zeros_like(s_channel)
    binary[(s_binary == 1) & (v_binary == 1)] = 1

    return binary

def region_of_interest(img, vertices=None):
    '''
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    '''
    
    if vertices == None:
        vertices = np.array([[bottom_left, (top_left[0], top_left[1]), (top_right[0], top_right[1]), bottom_right]], dtype=np.int32)

    # Defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    # Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Returning the image only where mask pixels are nonzero
    masked = cv2.bitwise_and(img, mask)
    
    return masked

def undistort(img):
    '''
    Performs the image distortion correction.
    
    Loads distortion correction information from pickle file.
    '''
    
    # Read in the saved obj_points and img_points
    dist_pickle = pickle.load(open(pickle_file, 'rb'))
    mtx = dist_pickle['mtx']
    dist = dist_pickle['dist']

    # Undistort the image
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
        
    return undistorted

def preprocess(img):
    '''
    Applies several thresholds together.
    
    Creates single channel image based on oriing image and thresholds.
    '''
    
    # Generate binaries from absolute Sobel x and y and color spaces
    grad_x = abs_sobel_thresh(img, orient='x', thresh=(12, 255)) # like canny transform
    grad_y = abs_sobel_thresh(img, orient='y', thresh=(25, 255))
    c_binary = color_thresh(img, s_thresh=(100, 255), v_thresh=(50, 255))

    # Create single channel image based on combination of Sobel and color spaces
    preprocessed = np.zeros_like(img[:, :, 0])
    preprocessed[(grad_x == 1) & (grad_y == 1) | (c_binary == 1)] = 255
    
    return preprocessed

def warp(img):
    '''
    Applies perspective transformation.
    
    This results in a top down view of the image.
    Transformation matrices are also created.
    '''
    
    # Define source and destination coordinates to apply transformation on
    src = np.float32([bottom_left, top_left, top_right, bottom_right])
    dst = np.float32([proj_bottom_left, proj_top_left, proj_top_right, proj_bottom_right])

    # Perform perspective transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src) # inverse matrix for later projection 
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv

def window_slide(img):
    # Assuming you have created a warped binary image called "img"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    windowed_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(windowed_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(windowed_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    windowed_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    windowed_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    return windowed_img, left_fit, left_fitx, right_fit, right_fitx, ploty
   
def polynomial_fit(img, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "img")
    # It's now much easier to find line pixels!
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, left_fitx, right_fit, right_fitx, ploty
    
def projection(img, warped, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    return result


def finalize(img, left_fit, left_fitx, right_fitx, ploty, ym_per_pix, xm_per_pix):
    # curvature
    leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    y_eval = np.max(ploty)
    curve_rad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])

    # offset of center
    camera_center = (left_fitx[-1] + right_fitx[-1]) / 2. # -1 to have the closest to the car
    center_diff = (camera_center - img.shape[1] / 2) * xm_per_pix

    # draw the text showing curvature, offset of center
    cv2.putText(img, 'Radius of Curvature: ' + str(round(curve_rad, 3)) + '(m)', (50, 50), cv2.FONT_ITALIC, 1, (255, 255, 255), 2)
    cv2.putText(img, 'Vehicle position of center: ' + str(round(center_diff, 3)) + '(m)', (50, 100), cv2.FONT_ITALIC, 1, (255, 255, 255), 2)

    return img


In [7]:
def process_frame(img, save_ouput_file=False):
    global previous_frame
    img_size = (img.shape[1], img.shape[0])
    
    # Perform distortion correction
    undistorted = undistort(img)
    # save the undistorted
    if save_ouput_file == True:
        save_output_file(undistorted, 'undistorted' + str(idx))

    # Perform threshold 
    preprocessed = preprocess(undistorted)
    # save the preprocessed
    if save_ouput_file == True:
        save_output_file(preprocessed, 'preprocessed' + str(idx))
        
    # Apply region of interest
    masked = region_of_interest(preprocessed)
    # save the masked
    if save_ouput_file == True:
        save_output_file(masked, 'masked' + str(idx))
        
    # Apply perspective transformation
    warped, M, Minv = warp(masked)
    # save the warped
    if save_ouput_file == True:
        save_output_file(warped, 'warped' + str(idx))

    # Find lines based on polynomial fit in case we have first frame
    # In the next frames we do not need to search from scratch
    if previous_frame is None:
        windowed, left_fit, left_fitx, right_fit, right_fitx, ploty = window_slide(warped)
        previous_frame = (left_fit, right_fit)
        
        # save the windowed
        if save_ouput_file == True:
            save_output_file(windowed, 'windowed' + str(idx)) 

    else:
        left_fit, left_fitx, right_fit, right_fitx, ploty = polynomial_fit(warped, previous_frame[0], previous_frame[1])
        previous_frame = (left_fit, right_fit)

    # Project lane on undistorted image
    projected = projection(undistorted, warped, left_fitx, right_fitx, ploty, Minv)

    # Apply curveture and offset of center position calculation
    result = finalize(projected, left_fit, left_fitx, right_fitx, ploty, conv_y, conv_x)
    # save the result
    if save_ouput_file == True:
        save_output_file(result, 'result' + str(idx)) 
    
    return result

## Read in image and apply pipeline on it

In [8]:
# Iterate through all test images
for idx, fname in enumerate(test_images_list):
    # read the image
    img = cv2.imread(fname)
    process_frame(img, True)

## Read in video and apply pipeline on it

In [9]:
clip = VideoFileClip(input_video)
video_clip = clip.fl_image(process_frame)
video_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video output_video.mp4
[MoviePy] Writing video output_video.mp4


100%|█████████▉| 1260/1261 [02:03<00:00, 10.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video.mp4 

